In [ ]:
import json
import os
import re

# 创建目录
NECESSARY_DIRS = [
    './dataset/java',
    './dataset/python',
    './dataset/repo',
    './enre_out/java', 
    './enre_out/java', 
]

os.makedirs('./dataset/java', exist_ok=True)

In [6]:
# 提取各个数据集的仓库 URL，已有忽略

# 根据使用修改：存放 CoEdPilot 训练集的目录
base_folder = r"C:\Users\aaa\Desktop\dataset&model\dataset&model\locator_data"

def commit_url_to_repo_url(commit_url):
    return re.search(r"(.*?)/commit/\w+$", commit_url)[1]

# 提取 base_folder 中 from_path 数据集的后缀为 lang_suffix 的文件的仓库地址，存放到 out_path
def extract_distinct_repo_url(from_path, to_path, lang_suffix):
    distinct_repo_url = set()
    with open(os.path.join(base_folder, from_path), 'r', encoding="utf-8") as file:
        lines = file.readlines()
        for line in lines:
            json_data = json.loads(line)
            if json_data["file_name"].endswith(f".{lang_suffix}"):
                distinct_repo_url.add(commit_url_to_repo_url(json_data["html_url"]))
    distinct_repo_url = list(distinct_repo_url)

    with open(to_path, "w", encoding="utf-8") as f:
        json.dump(distinct_repo_url, f)

# 为指定 lang 提取仓库地址
def extract_distinct_repo_url_for_lang(lang, lang_suffix):
    extract_distinct_repo_url(f"{lang}/test.jsonl", f"report_url/{lang}_repo_urls.json", lang_suffix)

In [ ]:
extract_distinct_repo_url_for_lang("python", "py")

In [1]:
# 运行对应语言的 ENRE 程序，并制作数据集

import make_nn_dataset_for_lang
import importlib
importlib.reload(make_nn_dataset_for_lang)
import json

# 为指定语言运行 make_dataset
def make_dataset_for_lang(lang):
    repos = []
    with open(f"repo_url/{lang}_repo_urls.json", "r", encoding="utf-8") as f:
        repos = json.load(f)

    for repo_url in repos:
        make_nn_dataset_for_lang.make_dataset(repo_url, lang)

In [ ]:
make_dataset_for_lang("python")